In [ ]:
!gdown 18BneEzOCDx6pKrDBQ61_vL0gdlsIguHG # mockup_ecomm_prod_cat.csv
!gdown 1YBptaYjlbLkMnOxJRa-iW4eJHPP2tZiT # mockup_ecomm_vend.csv
!gdown 1KEZhs5dngtRbEB9ULcxm_8-ua3RPX5lP # mockup_ecomm_prod.csv
!gdown 1o14PXP9xXD8gmfSvQpYFUKPJKNCEt2sv # mockup_ecomm_cust.csv
!gdown 1wvBuliOlE-Row73kK6Z2Hu4LC1LU7M7E # mockup_ecomm_promo.csv
!gdown 1O_FIQhP2-RCNxoGVymS6spLXf5mx6Ocd # mockup_ecomm_prod_promo.csv
!gdown 1jBOrx8Z3VfBfkAEe1omHplSs5mvRfIEX # mockup_ecomm_promo_redeem.csv
!gdown 11ynCSPn3Hqd9IlElH0VBCE07PrwKKxwT # mockup_ecomm_trx_header.csv
!gdown 1uBFJGtEfL9fny1vfszIo-IS67u2eLe1r # mockup_ecomm_trx_item.csv
!gdown 1D6K1L35enPB9_1u1l2Lc66ZVOxRBui-t # mockup_ecomm_trx_promo.csv

In [ ]:
import pandas as pd;
import numpy as np;

In [ ]:
class Dataset_202303:
    CUTOFF_DATE=pd.Timestamp('2023-04-01')

    prodx_cat:pd.DataFrame = None;
    vendors:pd.DataFrame = None;
    prodx:pd.DataFrame = None;
    customers:pd.DataFrame = None;

    promo:pd.DataFrame = None;
    prodx_promo:pd.DataFrame = None;
    promo_redeem:pd.DataFrame = None;

    trx_header:pd.DataFrame = None;
    trx_item:pd.DataFrame = None;
    trx_promo:pd.DataFrame = None;

    def __init__(self):
        self.prodx_cat:pd.DataFrame = pd.read_csv('mockup_ecomm_prod_cat.csv');
        self.vendors:pd.DataFrame = pd.read_csv('mockup_ecomm_vend.csv');
        self.prodx:pd.DataFrame = pd.read_csv('mockup_ecomm_prod.csv');
        self.customers:pd.DataFrame = pd.read_csv('mockup_ecomm_cust.csv');

        self.promo:pd.DataFrame = pd.read_csv('mockup_ecomm_promo.csv');
        self.prodx_promo:pd.DataFrame = pd.read_csv('mockup_ecomm_prod_promo.csv');
        self.promo_redeem:pd.DataFrame = pd.read_csv('mockup_ecomm_promo_redeem.csv');

        self.trx_header:pd.DataFrame = pd.read_csv('mockup_ecomm_trx_header.csv');
        self.trx_item:pd.DataFrame = pd.read_csv('mockup_ecomm_trx_item.csv');
        self.trx_promo:pd.DataFrame = pd.read_csv('mockup_ecomm_trx_promo.csv');
        pass;

    def get_prodx_cat(self) -> pd.DataFrame:
        df:pd.DataFrame = self.prodx_cat.loc[:];
        df['created_dt'] = df['created_dt'].apply(pd.Timestamp);
        df['updated_dt'] = df['updated_dt'].apply(pd.Timestamp);
        return df[ (df['updated_dt'] < self.CUTOFF_DATE) | (df['created_dt'] < self.CUTOFF_DATE) ].loc[:];

    def get_vendors(self) -> pd.DataFrame:
        df:pd.DataFrame = self.vendors.loc[:];
        df['registered_dt'] = df['registered_dt'].apply(pd.Timestamp);
        df['updated_dt'] = df['updated_dt'].apply(pd.Timestamp);
        return df[ (df['updated_dt'] < self.CUTOFF_DATE) | (df['registered_dt'] < self.CUTOFF_DATE) ].loc[:];

    def get_prodx(self) -> pd.DataFrame:
        df:pd.DataFrame = self.prodx.loc[:];
        df['created_dt'] = df['created_dt'].apply(pd.Timestamp);
        df['updated_dt'] = df['updated_dt'].apply(pd.Timestamp);

        df = df[ df['created_dt'] < self.CUTOFF_DATE ].loc[:]
        df['f'] = (df['updated_dt'] >= self.CUTOFF_DATE) & (df['created_dt'] < self.CUTOFF_DATE);
        df['price'] = df[['price', 'prev_price', 'f']].apply(lambda x: x['prev_price'] if x['f'] else x['price'], axis='columns');
        df['prev_price'] = df[['prev_price', 'f']].apply(lambda x: np.nan if x['f'] else x['prev_price'], axis='columns');
        df['updated_dt'] = df[['updated_dt', 'created_dt', 'f']].apply(lambda x: x['created_dt'] if x['f'] else x['updated_dt'], axis='columns');
        return df.drop(columns=['f']).loc[:];

    def get_customers(self) -> pd.DataFrame:
        df:pd.DataFrame = self.customers.loc[:];
        df['registered_dt'] = df['registered_dt'].apply(pd.Timestamp);
        df['updated_dt'] = df['updated_dt'].apply(pd.Timestamp);
        return df[ (df['updated_dt'] < self.CUTOFF_DATE) | (df['registered_dt'] < self.CUTOFF_DATE) ].loc[:];

    def get_promo(self) -> pd.DataFrame:
        df:pd.DataFrame = self.promo.loc[:];
        df['created_dt'] = df['created_dt'].apply(pd.Timestamp);
        df['updated_dt'] = df['updated_dt'].apply(pd.Timestamp);
        return df[ (df['updated_dt'] < self.CUTOFF_DATE) | (df['created_dt'] < self.CUTOFF_DATE) ].loc[:];

    def get_prodx_promo(self) -> pd.DataFrame:
        df:pd.DataFrame = self.prodx_promo.loc[:];
        df['created_dt'] = df['created_dt'].apply(pd.Timestamp);
        df['updated_dt'] = df['updated_dt'].apply(pd.Timestamp);
        return df[ (df['updated_dt'] < self.CUTOFF_DATE) | (df['created_dt'] < self.CUTOFF_DATE) ].loc[:];

    def get_promo_redeem(self) -> pd.DataFrame:
        df:pd.DataFrame = self.promo_redeem.loc[:];
        df['created_dt'] = df['created_dt'].apply(pd.Timestamp);
        df['activated_dt'] = df['activated_dt'].apply(pd.Timestamp);
        df['activated_dt'] = df['activated_dt'].apply(lambda x: x if x < self.CUTOFF_DATE else pd.NaT);
        return df[ df['created_dt'] < self.CUTOFF_DATE ].loc[:];

    def get_trx_header(self) -> pd.DataFrame:
        df:pd.DataFrame = self.trx_header.loc[:];
        df['timestamp'] = df['timestamp'].apply(pd.Timestamp);
        return df[ df['timestamp'] < self.CUTOFF_DATE ].loc[:];

    def get_trx_item(self) -> pd.DataFrame:
        df:pd.DataFrame = self.trx_item.loc[:];
        tx_hdr:pd.DataFrame = self.get_trx_header()[['id']];
        return df.drop(df[ ~df['trx_header_id'].isin(tx_hdr['id']) ].index);

    def get_trx_promo(self) -> pd.DataFrame:
        df:pd.DataFrame = self.trx_promo.loc[:];
        tx_hdr:pd.DataFrame = self.get_trx_header()[['id']];
        return df.drop( df[ ~df['trx_header_id'].isin(tx_hdr['id']) ].index );

    def extract_csv(self):
        self.get_prodx_cat().to_csv('mockup_ecomm_prod_cat-202303.csv', index=False);
        self.get_vendors().to_csv('mockup_ecomm_vend-202303.csv', index=False);
        self.get_prodx().to_csv('mockup_ecomm_prod-202303.csv', index=False);
        self.get_customers().to_csv('mockup_ecomm_cust-202303.csv', index=False);

        self.get_promo().to_csv('mockup_ecomm_promo-202303.csv', index=False);
        self.get_prodx_promo().to_csv('mockup_ecomm_prod_promo-202303.csv', index=False);
        self.get_promo_redeem().to_csv('mockup_ecomm_promo_redeem-202303.csv', index=False);

        self.get_trx_header().to_csv('mockup_ecomm_trx_header-202303.csv', index=False);
        self.get_trx_item().to_csv('mockup_ecomm_trx_item-202303.csv', index=False);
        self.get_trx_promo().to_csv('mockup_ecomm_trx_promo-202303.csv', index=False);
        pass;

    def download_csv(self):
        from google.colab import files;
        files.download('mockup_ecomm_prod_cat-202303.csv');
        files.download('mockup_ecomm_vend-202303.csv');
        files.download('mockup_ecomm_prod-202303.csv');
        files.download('mockup_ecomm_cust-202303.csv');

        files.download('mockup_ecomm_promo-202303.csv');
        files.download('mockup_ecomm_prod_promo-202303.csv');
        files.download('mockup_ecomm_promo_redeem-202303.csv');

        files.download('mockup_ecomm_trx_header-202303.csv');
        files.download('mockup_ecomm_trx_item-202303.csv');
        files.download('mockup_ecomm_trx_promo-202303.csv');
        pass;

dataset = Dataset_202303();
dataset.extract_csv();
# dataset.download_csv();

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>